# Necessary Imports and Data Readings


In [ ]:
import cv2
import numpy as np
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import glob

In [ ]:
work_dir = "/Users/emre/GitHub/HU-AI/AIN433/Assignment 1/"
DATASET_PATH = os.path.join(work_dir, "dataset")
IMAGES_PATH = os.path.join(DATASET_PATH, "images")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations")
RESULTS_PATH = os.path.join(work_dir, "results")

# Using glob to read image file names directly
png_img_list = [img for img in glob.glob(os.path.join(IMAGES_PATH, "*.png"))]

# File existence check is redundant if files are directly read into the list
print(f"Number of images: {len(png_img_list)}")

# Optimizing XML reading and parsing
xml_files = glob.glob(os.path.join(ANNOTATIONS_PATH, "*.xml"))
xml_coordinates = {
    os.path.join(IMAGES_PATH, ET.parse(xml_file).find("filename").text): [
        int(ET.parse(xml_file).find(".//xmin").text),
        int(ET.parse(xml_file).find(".//ymin").text),
        int(ET.parse(xml_file).find(".//xmax").text),
        int(ET.parse(xml_file).find(".//ymax").text),
    ]
    for xml_file in xml_files
}

# Sobel Edge Detector


In [ ]:
def sobel_edge_detector(image_path, kernel_size=3):
    """
    Apply Sobel edge detection to an image.

    Args:
    - image_path (str): Path to the input image.
    - kernel_size (int): Size of the Gaussian kernel for blurring.

    Returns:
    - Tuple of (grad_mag, grad_x, grad_y):
        - grad_mag (numpy.ndarray): Gradient magnitude image.
        - grad_x (numpy.ndarray): Gradient in the x-direction.
        - grad_y (numpy.ndarray): Gradient in the y-direction.
    """
    if kernel_size % 2 == 0 or kernel_size <= 1:
        raise ValueError("Kernel size must be odd and greater than 1.")

    # Read the image from the specified path
    img = cv2.imread(image_path)

    if img is None:
        raise FileNotFoundError(
            f"The specified image path does not exist: {image_path}"
        )

    # Convert the image to grayscale for edge detection
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image and reduce noise,
    # which helps in the edge detection process
    blurred_image = cv2.GaussianBlur(gray_image, (kernel_size, kernel_size), 0)

    # Define Sobel kernels for x and y directions
    kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    kernel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Apply convolution with the Sobel kernels to detect edges in x and y directions
    grad_x = cv2.filter2D(blurred_image, cv2.CV_64F, kernel_x)
    grad_y = cv2.filter2D(blurred_image, cv2.CV_64F, kernel_y)

    # Calculate the gradient magnitude as the Euclidean norm of gradients in x and y directions
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)

    # Convert gradient magnitude to 8-bit unsigned integer type for displaying
    # grad_mag_uint8 = np.array(grad_mag, dtype=np.uint8)
    grad_mag_uint8 = cv2.convertScaleAbs(grad_mag)

    return grad_mag_uint8, grad_x, grad_y

In [ ]:
# Use the function and display the result
sobel_img, x, y = sobel_edge_detector(png_img_list[0], kernel_size=3)

# Displaying the Sobel edge detection result
plt.figure(figsize=(4, 3))
plt.imshow(sobel_img, cmap="gray")
plt.axis("off")  # Hides the axis
plt.title("Sobel Edge Detection Result")
plt.show()

# Canny Edge Detector


In [ ]:
def non_maximum_suppression(gradient_magnitude, gradient_direction):
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros_like(gradient_magnitude)
    angle = gradient_direction * 180.0 / np.pi
    angle[angle < 0] += 180

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            q = 255
            r = 255

            # East-West (0 degrees)
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = gradient_magnitude[i, j + 1]
                r = gradient_magnitude[i, j - 1]
            # Northeast-Southwest (45 degrees)
            elif 22.5 <= angle[i, j] < 67.5:
                q = gradient_magnitude[i + 1, j - 1]
                r = gradient_magnitude[i - 1, j + 1]
            # North-South (90 degrees)
            elif 67.5 <= angle[i, j] < 112.5:
                q = gradient_magnitude[i + 1, j]
                r = gradient_magnitude[i - 1, j]
            # Northwest-Southeast (135 degrees)
            elif 112.5 <= angle[i, j] < 157.5:
                q = gradient_magnitude[i - 1, j - 1]
                r = gradient_magnitude[i + 1, j + 1]

            if (gradient_magnitude[i, j] >= q) and (gradient_magnitude[i, j] >= r):
                suppressed[i, j] = gradient_magnitude[i, j]
            else:
                suppressed[i, j] = 0

    return suppressed

In [ ]:
def double_threshold(image, low_threshold, high_threshold):
    high_threshold = image.max() * high_threshold
    low_threshold = high_threshold * low_threshold

    rows, cols = image.shape
    result = np.zeros((rows, cols), dtype=np.int32)

    weak = np.int32(25)
    strong = np.int32(255)

    strong_i, strong_j = np.where(image >= high_threshold)
    zeros_i, zeros_j = np.where(image < low_threshold)

    weak_i, weak_j = np.where((image <= high_threshold) & (image >= low_threshold))

    result[strong_i, strong_j] = strong
    result[weak_i, weak_j] = weak

    return result

In [ ]:
def edge_tracking(image):
    weak = 25
    strong = 255

    rows, cols = image.shape

    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            if image[i, j] == weak:
                if np.any(image[i - 1 : i + 2, j - 1 : j + 2] == strong) or np.any(
                    image[i - 1 : i + 2, j - 1 : j + 2] == strong
                ):
                    image[i, j] = strong
                else:
                    image[i, j] = 0
    return image

In [ ]:
def canny_edge_detector(image_path, kernel_size, low_threshold, high_threshold):
    # Compute Sobel gradients
    gradient_mag, sobelx, sobely = sobel_edge_detector(image_path)
    gradient_dir = np.arctan2(sobely, sobelx)

    # Non-maximum suppression
    suppressed = non_maximum_suppression(gradient_mag, gradient_dir)

    # Double thresholding
    thresholded = double_threshold(suppressed, low_threshold, high_threshold)

    # Edge tracking by hysteresis
    edges = edge_tracking(thresholded)

    return edges.astype("uint8")

In [ ]:
# Path to the input image
image_path = png_img_list[0]

# Perform Canny edge detection
canny_edges = canny_edge_detector(
    image_path, kernel_size=5, low_threshold=0.1, high_threshold=0.5
)

fig, ax = plt.subplots(figsize=(8, 6))  # Change the size as needed

# Display the image
ax.imshow(canny_edges, cmap="gray")
ax.axis("off")  # Turn off axis
plt.show()

# Utilize Hough Transform With Canny & Sobel


In [ ]:
def my_hough_lines(image, rho_resolution, theta_resolution, threshold):
    # Extract the height and width of the image
    height, width = image.shape

    # Define the maximum possible distance in the image (diagonal)
    max_rho = int(np.ceil(np.sqrt(height**2 + width**2)))

    # Define the range of theta from 0 to 180 degrees
    thetas = np.linspace(0, np.pi, int(np.round(np.pi / theta_resolution)))

    # Initialize the accumulator array
    accumulator = np.zeros((2 * max_rho, len(thetas)), dtype=np.uint64)

    # Find the indices of edge pixels in the image
    edge_indices = np.argwhere(image > 0)

    # Iterate over each edge pixel
    for y, x in edge_indices:
        # Iterate over each theta value
        for theta_index, theta in enumerate(thetas):
            # Calculate rho for the given theta
            rho = int(np.round(x * np.cos(theta) + y * np.sin(theta)))
            # Increment the corresponding accumulator cell
            accumulator[rho + max_rho, theta_index] += 1

    # Find the indices of cells with votes above the threshold
    rho_indices, theta_indices = np.where(accumulator >= threshold)

    # Extract the rho and theta values from the indices
    rhos = rho_indices - max_rho
    thetas = thetas[theta_indices]

    # Return the detected lines as (rho, theta) pairs
    return list(zip(rhos, thetas))

In [ ]:
def hough_transform(img_name, method="Canny"):
    # Read the input image
    image = cv2.imread(img_name)

    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform edge detection using the Canny edge detector
    if method == "Canny":
        edges = canny_edge_detector(img_name, 5, 50, 150)
    else:
        edges, _, _ = sobel_edge_detector(img_name, kernel_size=5)

    lines = my_hough_lines(edges, 1, np.pi / 180, 150)

    x_lines = [0, image.shape[0]]
    y_lines = [0, image.shape[1]]

    try:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 4)
            if x1 < 0 and x2 > image.shape[0]:
                y_lines.append((y1 + y2) // 2)

            elif y1 < 0 and y2 > image.shape[1]:
                x_lines.append((x1 + x2) // 2)

        x1, y1, x2, y2 = xml_coordinates[img_name]

        box_image = cv2.imread(img_name)
        box_image = cv2.cvtColor(box_image, cv2.COLOR_BGR2RGB)

        # Create a figure with two subplots
        fig, axes = plt.subplots(1, 3, figsize=(12, 6))
        # Display extracted edges
        axes[0].imshow(edges, cmap="gray")
        axes[0].set_title("Edge Extracted")
        axes[0].axis("off")

        # Display Hough lines
        axes[1].imshow(image)
        axes[1].set_title("Hough Lines")
        axes[1].axis("off")

        # Display Boundary Boxes
        axes[2].imshow(box_image)
        axes[2].set_title("Boundary Boxes")
        axes[2].axis("off")

    except TypeError:
        pass
    finally:
        x_lines = sorted(x_lines)
        y_lines = sorted(y_lines)

        return x_lines, y_lines, xml_coordinates[img_name]

# Predict Plate Boundaries


In [ ]:
def predict_plate(img_name, method="Canny"):
    # Read the input image
    image = cv2.imread(img_name)

    # Convert BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    x_lines, y_lines, real_coordinates = hough_transform(img_name, method)
    x_top = 0
    x_bottom = 10000

    y_top = 0
    y_bottom = 10000

    if len(x_lines) < 4:
        x_top = x_lines[0]
        x_bottom = x_lines[-1]

    else:
        x_top = x_lines[1]
        x_bottom = x_lines[-2]

    if len(y_lines) < 4:
        y_top = y_lines[0]
        y_bottom = y_lines[-1]

    else:
        y_top = y_lines[1]
        y_bottom = y_lines[-2]

    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    # Display extracted edges
    axes[0].imshow(image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    # Draw the vertical line
    plt.axvline(x=x_top, color="red")
    plt.axvline(x=x_bottom, color="red")

    # Draw the vertical line
    plt.axhline(y=y_top, color="red")
    plt.axhline(y=y_bottom, color="red")

    # Fill the area inside the box
    plt.fill_betweenx([y_top, y_bottom], x_top, x_bottom, color="red", alpha=0.5)

    x1, y1, x2, y2 = real_coordinates

    # Calculate coordinates for the intersection rectangle
    x_intersection_top = max(x_top, x1)
    y_intersection_top = max(y_top, y1)
    x_intersection_bottom = min(x_bottom, x2)
    y_intersection_bottom = min(y_bottom, y2)

    # Calculate coordinates for the union rectangle
    x_union_top = min(x_top, x1)
    y_union_top = min(y_top, y1)
    x_union_bottom = max(x_bottom, x2)
    y_union_bottom = max(y_bottom, y2)

    # Draw the first rectangle
    plt.plot(
        [x_top, x_bottom, x_bottom, x_top, x_top],
        [y_top, y_top, y_bottom, y_bottom, y_top],
        color="blue",
    )

    # Draw the second rectangle
    plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color="green")

    # Draw the intersection rectangle
    plt.fill_betweenx(
        [y_intersection_top, y_intersection_bottom],
        x_intersection_top,
        x_intersection_bottom,
        color="black",
        alpha=0.9,
    )

    # Draw the union rectangle
    plt.fill_betweenx(
        [y_union_top, y_union_bottom],
        x_union_top,
        x_union_bottom,
        color="yellow",
        alpha=0.3,
    )

    # Display extracted edges
    axes[1].imshow(image)
    axes[1].set_title("Boxes")
    axes[1].axis("off")

    # Show the image with the vertical line
    plt.show()

    # Calculate areas of the rectangles and the intersection
    area_rect1 = (x_bottom - x_top) * (y_bottom - y_top)
    area_rect2 = (x2 - x1) * (y2 - y1)
    area_intersection = max(0, x_intersection_bottom - x_intersection_top) * max(
        0, y_intersection_bottom - y_intersection_top
    )

    # Calculate area of the union
    area_union = area_rect1 + area_rect2 - area_intersection

    # Calculate Intersection over Union (IoU)
    iou = area_intersection / area_union

    return iou

# Calculate Overall Prediction


In [ ]:
canny_iou_values = list()

for image in png_img_list:
    iou = predict_plate(image, method="Canny")
    canny_iou_values.append(iou)
    print("Intersection over Union (IoU):", iou)

In [ ]:
print("Canny IoU Average : ", sum(canny_iou_values) / len(canny_iou_values))

In [ ]:
sobel_iou_values = list()

for image in png_img_list:
    iou = predict_plate(image, method="Sobel")
    sobel_iou_values.append(iou)
    print("Intersection over Union (IoU):", iou)

In [ ]:
print("Sobel IoU Average  : ", sum(sobel_iou_values) / len(sobel_iou_values))

In [ ]:
iou = predict_plate(png_img_list[0], method="Sobel")